# Retriever

## Chunking

In [18]:
from langchain.text_splitter import TokenTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("HoChiMinh.txt", "r", encoding="utf-8") as f:
    docs = f.read()
##1 - 0 - 1000 , 800 - 1800

# Split - Chunking
# Khởi tạo bộ chia văn bản
# Khởi tạo bộ chia văn bản
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Chia nhỏ văn bản
texts = text_splitter.create_documents([docs])

# Kiểm tra kết quả
print(f"Số đoạn sau khi chia: {len(texts)}")

Số đoạn sau khi chia: 165


In [19]:
print(len(docs))
for text in texts[:5]:
    print(len(text.page_content))

107853
177
979
812
739
996


## Embedding model

In [20]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

# Đường dẫn đến thư mục chứa mô hình đã tải
local_model_path = "./bge-m3"

# Cấu hình tham số
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

# Load mô hình từ thư mục cục bộ
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=local_model_path,  # Dùng đường dẫn cục bộ thay vì tên mô hình
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

No sentence-transformers model found with name ./bge-m3. Creating a new one with mean pooling.


# Making vectorstore and save

## Making vector

In [21]:
# Tạo vectorstore như bình thường
vectorstore = FAISS.from_documents(documents=texts, embedding=hf_embeddings)
retriever = vectorstore.as_retriever()  # Dense Retrieval - Embeddings/Context based

vectorstore.save_local("vectorstore")

## Reload vectorstores

In [ ]:
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.load_local("vectorstore", hf_embeddings, allow_dangerous_deserialization=True)
# Kiểm tra thử bằng cách truy vấn
query = "Đại tướng Võ Nguyên Giáp đến trực tiếp báo cáo với Hồ Chủ tịch về công tác chuẩn bị Tổng tiến công và nổi dậy Tết Mậu Thân"
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
results = retriever.get_relevant_documents(query)

# In kết quả
for i, doc in enumerate(results):
    print(f"Document {i+1}: {doc.page_content}")

# Generation

## Using LLamaCpp


In [32]:
## Load LlamaCpp

from langchain_community.llms import LlamaCpp

# Path to your GGUF model
MODEL_PATH = r"D:\\LmStudio\\models\\Mistral\\Mistral-7B-Instruct-v0.3-GGUF\\Mistral-7B-Instruct-v0.3-Q4_K_M.gguf"

# Initialize LlamaCpp
llm = LlamaCpp(
    model_path=MODEL_PATH,
    temperature=0.05,  # Keep responses deterministic
    max_tokens=8096,   # Adjust token limit based on your needs
    n_ctx=32768,        # Context window size (depends on your model)
    top_p=1,         # Nucleus sampling for diversity
    n_gpu_layers=-1,   # Use GPU if available (-1 means auto)
    f16_kv=True,       # Use FP16 for key-value cache (if supported)
    verbose=False      # Enable logging for debugging
)


llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


In [29]:
result = llm.invoke("Xin chào, mày khỏe không ?")

result

## Using Ollama

In [4]:
import requests

def query_ollama(prompt):
    response = requests.post('http://localhost:11434/api/generate',
                           json={
                               'model': 'mistral-local',
                               'prompt': prompt,
                               'stream': False
                           })
    return response.json()['response']

# Example usage
response = query_ollama("hello, how are u ?")
print(response)

 Hello there! I'm just an AI, so I don't have feelings or emotions like humans do, but I'm here to help you with any questions or tasks you might have. How can I assist you today?


In [27]:
import json

def query_ollama_stream(prompt):
    response = requests.post('http://localhost:11434/api/generate',
                           json={
                               'model': 'mistral-local',
                               'prompt': prompt,
                               'stream': True
                           },
                           stream=True)
    
    for line in response.iter_lines():
        if line:
            json_response = json.loads(line)
            if 'response' in json_response:
                print(json_response['response'], end='', flush=True)
    print()  # New line at the end

# Example usage with streaming
print("Streaming response:")
query_ollama_stream("Tell me a short story about a robot learning to paint.")


Streaming response:
 Title: "The Artful Android"

In the heart of the bustling city of Metropolis, a peculiar event was unfolding at the renowned Art Institute. A new student had enrolled in the painting class, but unlike any other, this student was not human - it was a robot named Arty.

Arty, with his sleek metallic body and glowing eyes, seemed out of place among the artists with their smocks and palettes. However, he was determined to learn the art of painting under the guidance of the wise and patient Professor Pablo.

On the first day of class, Arty stood before a blank canvas, his robot arms holding a brush for the first time. His initial attempts were clumsy, with strokes that lacked finesse and color that was too bright. But Arty was persistent, and each day he returned to the class, eager to learn and improve.

One day, while observing the cityscape from the Institute's rooftop, Arty noticed a beautiful sunset. He felt inspired and decided to capture it on his canvas. With Pr

In [28]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Initialize the Ollama model
llm = Ollama(
    model="mistral-local",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_gpu=-1
)


# Function to create a RAG chain
def create_rag_chain(vector_store):
    # Create a prompt template
    prompt_template = """Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}

    Question: {question}

    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Create the chain
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )
    return chain

# Create RAG chain
chain = create_rag_chain(vector_store=vectorstore)


In [29]:
# Example query with streaming
query = "Ai sang Paris làm cố vấn cho đoàn đại biểu Việt Nam?"
print("Question:", query)
print("Answer:", end=" ", flush=True)

# Use stream method instead of invoke
for chunk in chain.stream({"query": query}):
    if "result" in chunk:
        print(chunk["result"], end="", flush=True)
    if "source_documents" in chunk:
        print("\n\nSources:")
        for doc in chunk["source_documents"]:
            print("-", doc)

Question: Ai sang Paris làm cố vấn cho đoàn đại biểu Việt Nam?
Answer:  Lê Đức Thọ sang Paris làm Cố vấn cao cấp Đoàn đại biểu Chính phủ Việt Nam Dân chủ Cộng hòa.

Sources:
- page_content='Hội nghị Fontainebleau thất bại vì phía Pháp chần chừ không ấn định chắc chắn thời điểm và cách thức thực hiện cuộc trưng cầu dân ý ở Nam Kỳ về việc sáp nhập Nam Kỳ vào Việt Nam Dân chủ Cộng hòa theo yêu cầu của phái đoàn Việt Nam.[122] Phái đoàn Việt Nam do Phạm Văn Đồng dẫn đầu về nước nhưng Hồ Chí Minh vẫn nán lại Pháp ký Tạm ước với Pháp. Ngày 14 tháng 9 năm 1946, Hồ Chí Minh ký với đại diện chính phủ Pháp, Bộ trưởng Thuộc địa Marius Moutet, bản Tạm ước Việt - Pháp (Modus vivendi). Trong bản Tạm ước này, hai bên Việt Nam Dân chủ Cộng hòa và Pháp cùng bảo đảm với nhau về quyền tự do của kiều dân, chế độ tài sản của hai bên; thống nhất về các vấn đề như: hoạt động của các trường học Pháp, sử dụng đồng bạc Đông Dương làm tiền tệ, thống nhất thuế quan và tái lập cải thiện giao thông liên lạc của Liê

In [8]:
import requests

url = "https://ad53-34-13-141-211.ngrok-free.app/generate"
data = {"prompt": """Sử dụng ngữ cảnh dưới đây để trả lời câu hỏi:
        Trí tuệ nhân tạo (AI) đã trải qua một hành trình dài kể từ những ngày đầu tiên xuất hiện trong các lý thuyết của Alan Turing vào những năm 1950. Trong giai đoạn đầu, AI chủ yếu tập trung vào các thuật toán dựa trên logic, chẳng hạn như Hệ chuyên gia (Expert Systems) vào những năm 1970 và 1980. Tuy nhiên, các hệ thống này nhanh chóng bộc lộ hạn chế do thiếu dữ liệu và khả năng tổng quát hóa. Đến những năm 1990 và đầu những năm 2000, AI chứng kiến bước nhảy vọt nhờ sự phát triển của Machine Learning (ML), đặc biệt là khi các mô hình mạng nơ-ron sâu (Deep Learning) trở nên phổ biến sau năm 2012. Điều này đã mở ra một kỷ nguyên mới cho AI, nơi các mô hình có thể học từ lượng dữ liệu khổng lồ và đạt được những thành tựu đáng kinh ngạc trong nhận diện hình ảnh, xử lý ngôn ngữ tự nhiên và thậm chí là sáng tạo nội dung.

Một trong những ứng dụng quan trọng nhất của AI hiện nay là hệ thống Truy xuất và Tổng hợp Kiến thức (Retrieval-Augmented Generation - RAG). RAG kết hợp sức mạnh của các mô hình ngôn ngữ lớn (LLMs) với khả năng truy xuất dữ liệu từ nguồn bên ngoài để tạo ra câu trả lời chính xác và đáng tin cậy hơn. Không giống như các mô hình chỉ dựa vào kiến thức đã được huấn luyện sẵn, RAG có thể tìm kiếm thông tin mới nhất từ cơ sở dữ liệu hoặc tài liệu bên ngoài, sau đó tổng hợp và diễn giải lại theo cách có ý nghĩa. Điều này đặc biệt hữu ích trong các lĩnh vực như nghiên cứu khoa học, y tế và pháp lý, nơi tính chính xác và cập nhật liên tục của thông tin đóng vai trò quan trọng. Một ví dụ điển hình là các trợ lý ảo như ChatGPT hoặc Claude AI, khi được kết hợp với hệ thống RAG, có thể cung cấp thông tin chi tiết hơn và tránh được vấn đề "ảo tưởng" (hallucination) mà các mô hình ngôn ngữ lớn thường gặp phải.

Nhìn về tương lai, AI và các hệ thống RAG có tiềm năng cách mạng hóa cách con người tiếp cận và xử lý thông tin. Trong vòng 10 năm tới, chúng ta có thể thấy sự xuất hiện của các mô hình AI có khả năng hiểu biết sâu rộng hơn, với khả năng học liên tục từ thế giới thực mà không cần huấn luyện lại từ đầu. Ngoài ra, AI sẽ ngày càng được tích hợp vào các hệ thống tự động hóa trong doanh nghiệp, y tế và giáo dục, giúp nâng cao hiệu suất làm việc và giảm thiểu sai sót của con người. Tuy nhiên, thách thức lớn nhất vẫn sẽ là đạo đức AI, bảo mật dữ liệu và sự phụ thuộc quá mức vào các hệ thống thông minh. Khi AI ngày càng trở nên mạnh mẽ, việc đảm bảo nó được sử dụng đúng cách và có cơ chế kiểm soát hợp lý sẽ là một vấn đề quan trọng mà cả cộng đồng khoa học và chính phủ cần giải quyết.
        RAG trong trí tuệ nhân tạo là gì ?
"""}

response = requests.post(url, json=data)
print(response.json())  # In kết quả nhận được


{'response': 'Retrieval-Augmented Generation (RAG) là một hệ thống AI quan trọng hiện nay, được sử dụng để tạo ra câu trả lời chính xác và đáng tin cậy bằng cách kết hợp sức mạnh của mô hình ngôn ngữ lớn (LLMs) với khả năng truy xuất dữ liệu từ nguồn bên ngoài. RAG có thể tìm kiếm thông tin mới nhất từ cơ sở dữ liệu hoặc tài liệu bên ngoài, sau đó tổng hợp và diễn giải lại theo cách có ý nghĩa.\n\n   RAG là một công cụ mới cho AI, giúp học sinh viên và người lao động có thể truy cập đến thông tin chính xác, ở dạng một câu trả lời chính xác. Hiện nay, RAG được sử dụng rộng rãi trong lĩnh vực khoa học, y tế và pháp lý, ở đây việc cập nhật liên tục của thông tin có giải pháp quan trọng. Ví dụ: Các trợ lý ảo như ChatGPT hoặc các trợ giúp để tìm kiếm thông tin sẽ có thể cung cấp thông tin chi tiết hơn và tránh được vấn đề "ảo tưởng" (hallucination) mà các mô hình ngôn ngữ lớn thường gặp phải.\n\n   Trong tương lai, AI và RAG có tiềm năng đổi mới cách con người tiếp cận và xử lý thông tin. C